<a href="https://colab.research.google.com/github/archipelcitoyen/TAS_2026/blob/main/TAS_INSEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ANALYSE DES CARREAUX INSEE 2019 POUR LE CIBLAGE DU PORTE-À-PORTE - MUNICIPALES TOULOUSE 2026 - ARCHIPEL CITOYEN

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import requests
import zipfile
#import io
import os
import fiona
from urllib.request import urlretrieve
from shapely.geometry import box
#import urllib.request
import geopandas as gpd
import folium
from folium import Choropleth, LayerControl
import branca.colormap as cm

In [ ]:
# Configuration
DATA_DIR = "data"
os.makedirs(DATA_DIR, exist_ok=True)

In [ ]:
# 1. Téléchargement des données
def telecharger_donnees_insee():
    """Télécharge les données carroyées de l'INSEE à 200m"""
    
    print("Téléchargement des données carroyées INSEE 200m...")
    
    # URL de téléchargement des données carroyées 200m les plus récentes (2019)
    url_carreaux = "https://www.insee.fr/fr/statistiques/fichier/7655475/Filosofi2019_carreaux_200m_gpkg.zip"

    # Chemin local pour sauvegarder le fichier zip
    zip_path = os.path.join(DATA_DIR, "Filosofi2019_carreaux_200m_gpkg.zip")
    
    # Télécharger le fichier
    urlretrieve(url_carreaux, zip_path)
    
    # Extraire le contenu du zip
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(DATA_DIR)
    
    print("Téléchargement et extraction terminés.")

def charger_donnees_insee_local(chemin_geopackage):
    """Charge les données carroyées de l'INSEE à partir d'un fichier geopackage local"""
    
    print(f"Chargement des données carroyées INSEE 200m depuis le fichier local : {chemin_geopackage}")
    
    # Vérification que le fichier existe
    if not os.path.exists(chemin_geopackage):
        raise FileNotFoundError(f"Le fichier {chemin_geopackage} n'existe pas.")
    
    # Chargement du geopackage
    try:
        # Lister les couches disponibles dans le geopackage
        layers = fiona.listlayers(chemin_geopackage)
        print(f"Couches disponibles dans le geopackage : {layers}")
        
        # Charger la première couche (ou spécifier le nom si connu)
        layer_name = layers[0]
        carreaux_gdf = gpd.read_file(chemin_geopackage, layer=layer_name)
        
        print(f"Données chargées avec succès. {len(carreaux_gdf)} carreaux trouvés.")
        print(f"CRS des données : {carreaux_gdf.crs}")
        print(f"Colonnes disponibles : {carreaux_gdf.columns.tolist()}")
        
        return carreaux_gdf
    
    except Exception as e:
        print(f"Erreur lors du chargement du geopackage : {e}")
        raise

In [ ]:

# # 2. Téléchargement des limites administratives de Toulouse
# def telecharger_limites_toulouse():
#     """Télécharge les limites administratives de Toulouse"""
    
#     print("Téléchargement des limites administratives de Toulouse...")
    
#     # Utilisation d'un fichier GeoJSON des limites communales à jour
#     # Source: data.gouv.fr avec les limites administratives de 2023
#     url_communes = "https://www.data.gouv.fr/fr/datasets/r/07b7c9a2-d1e2-4da6-9f20-01a7b72d4b12"
    
#     communes_path = os.path.join(DATA_DIR, "communes_31.geojson")
#     urlretrieve(url_communes, communes_path)
    
#     # Charger les communes et filtrer pour Toulouse
#     communes = gpd.read_file(communes_path)
#     toulouse = communes[communes['nom'] == 'TOULOUSE']
    
#     # Sauvegarder les limites de Toulouse
#     toulouse_path = os.path.join(DATA_DIR, "toulouse.geojson")
#     toulouse.to_file(toulouse_path, driver="GeoJSON")
    
#     print("Limites de Toulouse téléchargées et sauvegardées.")
#     return toulouse

# def telecharger_limites_toulouse(force_dl=False):
#     """
#     Télécharge et extrait les limites administratives de Toulouse depuis Admin Express.

#     Args:
#         force_dl (bool): Forcer le téléchargement même si le fichier existe déjà.

#     Returns:
#         GeoDataFrame: Limites administratives de Toulouse.
#     """

#     # URL du fichier GeoJSON des communes françaises (Admin Express)
#     url_communes = "https://static.data.gouv.fr/resources/contours-communes-france-administrative-format-admin-express-avec-arrondissements/20230719-140000/commune-frmetdrom.geojson"
#     communes_path = os.path.join(DATA_DIR, "communes_france.geojson")
#     toulouse_path = os.path.join(DATA_DIR, "toulouse.geojson")

#     # Télécharger le fichier si nécessaire
#     if not os.path.exists(communes_path) or force_dl:
#         print("🔽 Téléchargement des limites communales françaises...")
#         try:
#             urllib.request.urlretrieve(url_communes, communes_path)
#             print("✅ Téléchargement terminé.")
#         except Exception as e:
#             print("❌ Erreur lors du téléchargement :", e)
#             return None
#     else:
#         print("✅ Fichier des communes déjà présent.")

#     # Charger les communes et filtrer pour Toulouse
#     try:
#         communes = gpd.read_file(communes_path)
#         # Vérifier la présence de la colonne 'nom'
#         if 'nom' not in communes.columns:
#             print("❌ La colonne 'nom' est absente du fichier GeoJSON.")
#             return None
#         toulouse = communes[communes['nom'].str.upper() == 'TOULOUSE']
#         if toulouse.empty:
#             print("❌ La commune de Toulouse n'a pas été trouvée dans le fichier.")
#             return None
#         # Sauvegarder les limites de Toulouse
#         toulouse.to_file(toulouse_path, driver="GeoJSON")
#         print("✅ Limites de Toulouse sauvegardées.")
#         return toulouse
#     except Exception as e:
#         print("❌ Erreur lors du traitement du fichier GeoJSON :", e)
#         return None



In [ ]:

# 3. Traitement des données carroyées
def decouper_carreaux_toulouse(carreaux_gdf, toulouse_gdf):
    """Découpe les carreaux selon les limites de Toulouse"""
    print("Découpage des carreaux selon les limites de Toulouse...")
    # Reprojection des limites de Toulouse dans le même CRS que les carreaux
    toulouse_reproj = toulouse_gdf.to_crs(carreaux_gdf.crs)
    # Découpage des carreaux selon les limites de Toulouse
    carreaux_toulouse = gpd.sjoin(carreaux_gdf, toulouse_reproj, how="inner", predicate="intersects")
    # Sauvegarde des carreaux de Toulouse
    carreaux_toulouse_path = os.path.join(DATA_DIR, "carreaux_toulouse.gpkg")
    carreaux_toulouse.to_file(carreaux_toulouse_path, driver="GPKG")
    print(f"Données carroyées découpées. {len(carreaux_toulouse)} carreaux trouvés pour Toulouse.")
    return carreaux_toulouse

def calculer_indicateurs(carreaux):
    """
    Calcule les indicateurs de précarité pour chaque carreau.
    """
    # Densité de population (nombre d'individus par km²)
    if 'ind' in carreaux.columns:
        carreaux['densite_pop'] = carreaux['ind'] / 0.04  # 200m x 200m = 0.04 km²
    else:
        carreaux['densite_pop'] = np.nan

    # Pourcentage de locataires
    if 'men_prop' in carreaux.columns and 'men' in carreaux.columns:
        carreaux['pct_locataires'] = (1 - carreaux['men_prop'] / carreaux['men']) * 100
    else:
        carreaux['pct_locataires'] = np.nan

    # # Revenu moyen
    # if 'ind_r' in carreaux.columns:
    #     carreaux['revenu_moyen'] = carreaux['ind_r']
    # else:
    #     carreaux['revenu_moyen'] = np.nan

    # # Pourcentage de population sous le seuil de bas revenus ---> calcul à corriger si inclus
    # if 'ind_snv' in carreaux.columns and 'ind' in carreaux.columns:
    #     carreaux['pct_bas_revenus'] = (carreaux['ind_snv'] / carreaux['ind']) * 100
    # else:
    #     carreaux['pct_bas_revenus'] = np.nan

    # Pourcentage de ménages pauvres
    if 'men_pauv' in carreaux.columns and 'ind' in carreaux.columns:
        carreaux['pct_men_pauvres'] = (carreaux['men_pauv'] / carreaux['men']) * 100
    else:
        carreaux['pct_men_pauvres'] = np.nan

    # Pourcentage de familles monoparentales
    if 'men_fmp' in carreaux.columns and 'men' in carreaux.columns:
        carreaux['pct_fam_monoparentales'] = (carreaux['men_fmp'] / carreaux['men']) * 100
    else:
        carreaux['pct_fam_monoparentales'] = np.nan

    return carreaux

def normaliser_indicateurs(carreaux):
    """
    Normalise les indicateurs de précarité pour chaque carreau.
    """
    def normaliser(serie):
        min_val = serie.min()
        max_val = serie.max()
        if pd.notnull(min_val) and pd.notnull(max_val) and max_val > min_val:
            return (serie - min_val) / (max_val - min_val)
        else:
            return pd.Series([0] * len(serie), index=serie.index)

    # Normalisation
    carreaux['norm_densite_pop'] = normaliser(carreaux['densite_pop'])
    carreaux['norm_pct_locataires'] = normaliser(carreaux['pct_locataires'])
    #carreaux['norm_revenu_moyen'] = 1 - normaliser(carreaux['revenu_moyen'])  # Inversion car revenu élevé = moins de précarité
    #carreaux['norm_pct_bas_revenus'] = normaliser(carreaux['pct_bas_revenus'])
    carreaux['norm_pct_men_pauvres'] = normaliser(carreaux['pct_men_pauvres'])
    carreaux['norm_pct_fam_monoparentales'] = normaliser(carreaux['pct_fam_monoparentales'])

    return carreaux

def calculer_indicateur_composite(carreaux):
    """
    Calcule l'indicateur composite de précarité pour chaque carreau.
    """
    # Pondérations
    poids = {
        'norm_densite_pop': 0.2,
        'norm_pct_bas_revenus': 0.4,
        #'norm_revenu_moyen': 0.3,
        'norm_pct_locataires': 0.2,
        'norm_pct_fam_monoparentales': 0.2
    }

    # Calcul de l'indicateur composite
    indicateur = pd.Series(0, index=carreaux.index, dtype=float)
    poids_total = 0

    for indicateur_nom, p in poids.items():
        if indicateur_nom in carreaux.columns and not carreaux[indicateur_nom].isnull().all():
            indicateur += p * carreaux[indicateur_nom]
            poids_total += p

    if poids_total > 0:
        carreaux['indicateur_precarite'] = indicateur / poids_total
    else:
        carreaux['indicateur_precarite'] = 0

    return carreaux

def filtrer_carreaux(carreaux, seuil_population=20):
    """
    Filtre les carreaux ayant une population inférieure au seuil spécifié.
    """
    if 'ind' in carreaux.columns:
        return carreaux[carreaux['ind'] >= seuil_population]
    else:
        return carreaux


# def calculer_indicateur_precarite(carreaux_toulouse, chemin_sauvegarde):
#     """
#     Calcule l'indicateur composite de précarité pour les carreaux de Toulouse et sauvegarde le résultat.
#     """
#     print("Calcul de l'indicateur de précarité...")

#     carreaux = calculer_indicateurs(carreaux_toulouse)
#     carreaux = normaliser_indicateurs(carreaux)
#     carreaux = calculer_indicateur_composite(carreaux)
#     carreaux = filtrer_carreaux(carreaux)
#     sauvegarder_resultat(carreaux, chemin_sauvegarde)

#     print("Indicateur de précarité calculé et sauvegardé.")
#     return carreaux

# # 4. Calcul de l'indicateur composite de précarité
# def calculer_indicateur_precarite(carreaux_toulouse):
#     """Calcule un indicateur composite de précarité basé sur plusieurs variables"""
    
#     print("Calcul de l'indicateur de précarité...")
    
#     # Vérification des colonnes disponibles dans le geopackage
#     print(f"Colonnes disponibles : {carreaux_toulouse.columns.tolist()}")
    
#     # Création de l'indicateur de précarité
#     try:
#         # Calcul de la densité de population (si la colonne Ind existe)
#         if 'Ind' in carreaux_toulouse.columns:
#             carreaux_toulouse['densite_pop'] = carreaux_toulouse['Ind'] / 0.04  # carreaux de 200m x 200m = 0.04 km²
#         else:
#             # Chercher une colonne alternative pour la population
#             pop_cols = [col for col in carreaux_toulouse.columns if 'pop' in col.lower() or 'ind' in col.lower()]
#             if pop_cols:
#                 print(f"Utilisation de la colonne {pop_cols[0]} pour la population")
#                 carreaux_toulouse['densite_pop'] = carreaux_toulouse[pop_cols[0]] / 0.04
#             else:
#                 print("Aucune colonne de population trouvée, densité non calculée")
#                 carreaux_toulouse['densite_pop'] = np.nan
        
#         # Pourcentage de locataires (adaptation selon les colonnes disponibles)
#         if 'Men_prop' in carreaux_toulouse.columns and 'Men' in carreaux_toulouse.columns:
#             carreaux_toulouse['pct_locataires'] = (1 - carreaux_toulouse['Men_prop'] / carreaux_toulouse['Men']) * 100
#         elif 'prop' in carreaux_toulouse.columns and 'log' in carreaux_toulouse.columns:
#             carreaux_toulouse['pct_locataires'] = (1 - carreaux_toulouse['prop'] / carreaux_toulouse['log']) * 100
#         else:
#             # Chercher des colonnes alternatives
#             prop_cols = [col for col in carreaux_toulouse.columns if 'prop' in col.lower()]
#             men_cols = [col for col in carreaux_toulouse.columns if 'men' in col.lower() or 'log' in col.lower()]
            
#             if prop_cols and men_cols:
#                 print(f"Utilisation des colonnes {prop_cols[0]} et {men_cols[0]} pour le taux de locataires")
#                 carreaux_toulouse['pct_locataires'] = (1 - carreaux_toulouse[prop_cols[0]] / carreaux_toulouse[men_cols[0]]) * 100
#             else:
#                 print("Données de statut d'occupation non trouvées, taux de locataires non calculé")
#                 carreaux_toulouse['pct_locataires'] = np.nan
        
#         carreaux_toulouse['pct_locataires'] = carreaux_toulouse['pct_locataires'].fillna(0)
        
#         # Revenu moyen (adaptation selon les colonnes disponibles)
#         if 'Ind_r' in carreaux_toulouse.columns:
#             carreaux_toulouse['revenu_moyen'] = carreaux_toulouse['Ind_r']
#         else:
#             # Chercher une colonne alternative pour le revenu
#             rev_cols = [col for col in carreaux_toulouse.columns if 'rev' in col.lower() or 'r_' in col.lower()]
#             if rev_cols:
#                 print(f"Utilisation de la colonne {rev_cols[0]} pour le revenu")
#                 carreaux_toulouse['revenu_moyen'] = carreaux_toulouse[rev_cols[0]]
#             else:
#                 print("Données de revenu non trouvées, revenu moyen non calculé")
#                 carreaux_toulouse['revenu_moyen'] = np.nan
        
#         # Pourcentage de population sous le seuil de bas revenus
#         if 'Ind_snv' in carreaux_toulouse.columns and 'Ind' in carreaux_toulouse.columns:
#             carreaux_toulouse['pct_bas_revenus'] = carreaux_toulouse['Ind_snv'] / carreaux_toulouse['Ind'] * 100
#         else:
#             # Chercher des colonnes alternatives
#             snv_cols = [col for col in carreaux_toulouse.columns if 'snv' in col.lower() or 'bas' in col.lower()]
#             pop_cols = [col for col in carreaux_toulouse.columns if 'pop' in col.lower() or 'ind' in col.lower()]
            
#             if snv_cols and pop_cols:
#                 print(f"Utilisation des colonnes {snv_cols[0]} et {pop_cols[0]} pour le taux de bas revenus")
#                 carreaux_toulouse['pct_bas_revenus'] = carreaux_toulouse[snv_cols[0]] / carreaux_toulouse[pop_cols[0]] * 100
#             else:
#                 print("Données de bas revenus non trouvées, taux non calculé")
#                 carreaux_toulouse['pct_bas_revenus'] = np.nan
        
#         carreaux_toulouse['pct_bas_revenus'] = carreaux_toulouse['pct_bas_revenus'].fillna(0)
        
#         # Pourcentage de familles monoparentales
#         if 'Men_fmp' in carreaux_toulouse.columns and 'Men' in carreaux_toulouse.columns:
#             carreaux_toulouse['pct_fam_monoparentales'] = carreaux_toulouse['Men_fmp'] / carreaux_toulouse['Men'] * 100
#         else:
#             # Chercher des colonnes alternatives
#             fmp_cols = [col for col in carreaux_toulouse.columns if 'fmp' in col.lower() or 'mono' in col.lower()]
#             men_cols = [col for col in carreaux_toulouse.columns if 'men' in col.lower() or 'fam' in col.lower()]
            
#             if fmp_cols and men_cols:
#                 print(f"Utilisation des colonnes {fmp_cols[0]} et {men_cols[0]} pour le taux de familles monoparentales")
#                 carreaux_toulouse['pct_fam_monoparentales'] = carreaux_toulouse[fmp_cols[0]] / carreaux_toulouse[men_cols[0]] * 100
#             else:
#                 print("Données de familles monoparentales non trouvées, taux non calculé")
#                 carreaux_toulouse['pct_fam_monoparentales'] = np.nan
        
#         carreaux_toulouse['pct_fam_monoparentales'] = carreaux_toulouse['pct_fam_monoparentales'].fillna(0)
        
#         # Fonctions pour normaliser les variables
#         def normaliser(serie):
#             min_val = serie.min()
#             max_val = serie.max()
#             if max_val > min_val:
#                 return (serie - min_val) / (max_val - min_val)
#             else:
#                 return serie * 0
        
#         # Normalisation des variables
#         has_loc = not carreaux_toulouse['pct_locataires'].isna().all()
#         has_rev = not carreaux_toulouse['revenu_moyen'].isna().all()
#         has_basr = not carreaux_toulouse['pct_bas_revenus'].isna().all()
#         has_fmp = not carreaux_toulouse['pct_fam_monoparentales'].isna().all()
        
#         # Normalisation avec gestion des NaN
#         if has_loc:
#             carreaux_toulouse['norm_pct_locataires'] = normaliser(carreaux_toulouse['pct_locataires'])
#         else:
#             carreaux_toulouse['norm_pct_locataires'] = 0
            
#         if has_rev:
#             carreaux_toulouse['norm_revenu_moyen'] = 1 - normaliser(carreaux_toulouse['revenu_moyen'])
#         else:
#             carreaux_toulouse['norm_revenu_moyen'] = 0
            
#         if has_basr:
#             carreaux_toulouse['norm_pct_bas_revenus'] = normaliser(carreaux_toulouse['pct_bas_revenus'])
#         else:
#             carreaux_toulouse['norm_pct_bas_revenus'] = 0
            
#         if has_fmp:
#             carreaux_toulouse['norm_pct_fam_monoparentales'] = normaliser(carreaux_toulouse['pct_fam_monoparentales'])
#         else:
#             carreaux_toulouse['norm_pct_fam_monoparentales'] = 0
        
#         # Adaptation des pondérations selon les données disponibles
#         poids_total = 0
#         indicateur = 0
        
#         if has_basr:
#             indicateur += 0.4 * carreaux_toulouse['norm_pct_bas_revenus']
#             poids_total += 0.4
            
#         if has_rev:
#             indicateur += 0.3 * carreaux_toulouse['norm_revenu_moyen']
#             poids_total += 0.3
            
#         if has_loc:
#             indicateur += 0.2 * carreaux_toulouse['norm_pct_locataires']
#             poids_total += 0.2
            
#         if has_fmp:
#             indicateur += 0.1 * carreaux_toulouse['norm_pct_fam_monoparentales']
#             poids_total += 0.1
        
#         # Normalisation par le poids total pour avoir un indicateur entre 0 et 1
#         if poids_total > 0:
#             carreaux_toulouse['indicateur_precarite'] = indicateur / poids_total
#         else:
#             print("Attention : aucune donnée disponible pour calculer l'indicateur de précarité")
#             carreaux_toulouse['indicateur_precarite'] = 0
        
#         # Filtrer les carreaux avec une population suffisante
#         pop_seuil = 20
#         if 'Ind' in carreaux_toulouse.columns:
#             carreaux_toulouse = carreaux_toulouse[carreaux_toulouse['Ind'] >= pop_seuil]
#         elif 'pop' in carreaux_toulouse.columns:
#             carreaux_toulouse = carreaux_toulouse[carreaux_toulouse['pop'] >= pop_seuil]
#         else:
#             pop_cols = [col for col in carreaux_toulouse.columns if 'pop' in col.lower() or 'ind' in col.lower()]
#             if pop_cols:
#                 carreaux_toulouse = carreaux_toulouse[carreaux_toulouse[pop_cols[0]] >= pop_seuil]
        
#         # Sauvegarde du résultat
#         carreaux_precarite_path = os.path.join(DATA_DIR, "carreaux_toulouse_precarite.gpkg")
#         carreaux_toulouse.to_file(carreaux_precarite_path, driver="GPKG")
        
#         print("Indicateur de précarité calculé et sauvegardé.")
#         return carreaux_toulouse
        
#     except Exception as e:
#         print(f"Erreur lors du calcul de l'indicateur de précarité : {e}")
#         raise

# # 5. Sélection aléatoire de carreaux au-dessus d'un seuil
# def selectionner_carreaux_cibles(carreaux_toulouse, seuil_precarite=0.7, nb_carreaux=10):
#     """Sélectionne aléatoirement des carreaux dépassant un seuil de précarité"""
    
#     print(f"Sélection de {nb_carreaux} carreaux cibles avec seuil de précarité > {seuil_precarite}...")
    
#     # Filtrage des carreaux selon le seuil de précarité
#     carreaux_cibles = carreaux_toulouse[carreaux_toulouse['indicateur_precarite'] >= seuil_precarite]
    
#     if len(carreaux_cibles) == 0:
#         print(f"Aucun carreau ne dépasse le seuil de précarité {seuil_precarite}.")
#         print(f"La valeur maximale de l'indicateur est {carreaux_toulouse['indicateur_precarite'].max()}")
#         # On prend les 10% les plus précaires
#         seuil_ajuste = carreaux_toulouse['indicateur_precarite'].quantile(0.9)
#         print(f"Ajustement du seuil à {seuil_ajuste} (top 10%)")
#         carreaux_cibles = carreaux_toulouse[carreaux_toulouse['indicateur_precarite'] >= seuil_ajuste]
    
#     # Tirage aléatoire
#     if len(carreaux_cibles) > nb_carreaux:
#         carreaux_selectionnes = carreaux_cibles.sample(nb_carreaux, random_state=42)
#     else:
#         carreaux_selectionnes = carreaux_cibles
#         print(f"Seulement {len(carreaux_selectionnes)} carreaux disponibles au-dessus du seuil.")
    
#     # Sauvegarde des carreaux sélectionnés
#     carreaux_selectionnes_path = os.path.join(DATA_DIR, "carreaux_selectionnes.gpkg")
#     carreaux_selectionnes.to_file(carreaux_selectionnes_path, driver="GPKG")
    
#     print(f"{len(carreaux_selectionnes)} carreaux sélectionnés pour le porte-à-porte.")
#     return carreaux_selectionnes

# # 6. Création de cartes pour visualiser les résultats
# def creer_cartes(carreaux_toulouse, carreaux_selectionnes, toulouse_gdf):
#     """Crée des cartes pour visualiser les résultats"""
    
#     print("Création des cartes...")
    
#     # Reprojection en WGS84 pour un affichage plus standard
#     carreaux_toulouse_wgs84 = carreaux_toulouse.to_crs("EPSG:4326")
#     carreaux_selectionnes_wgs84 = carreaux_selectionnes.to_crs("EPSG:4326")
#     toulouse_wgs84 = toulouse_gdf.to_crs("EPSG:4326")
    
#     # Carte de l'indicateur de précarité
#     fig, ax = plt.subplots(figsize=(12, 10))
    
#     # Fond de carte avec les limites de Toulouse
#     toulouse_wgs84.boundary.plot(ax=ax, color='black', linewidth=1)
    
#     # Carte choroplèthe de l'indicateur de précarité
#     carreaux_toulouse_wgs84.plot(
#         column='indicateur_precarite',
#         ax=ax,
#         legend=True,
#         cmap='YlOrRd',
#         legend_kwds={'label': "Indicateur de précarité"},
#         alpha=0.7
#     )
    
#     # Surligner les carreaux sélectionnés
#     carreaux_selectionnes_wgs84.plot(
#         ax=ax,
#         color='none',
#         edgecolor='blue',
#         linewidth=1
#     )
    
#     # Titre et légende
#     plt.title('Indicateur de précarité par carreau à Toulouse')
#     plt.axis('off')
    
#     # Sauvegarde de la carte
#     carte_path = os.path.join(DATA_DIR, "carte_precarite_toulouse.png")
#     plt.savefig(carte_path, dpi=300, bbox_inches='tight')
    
#     print(f"Carte sauvegardée : {carte_path}")

# # 7. Exportation des données pour le porte-à-porte
# def exporter_donnees_porte_a_porte(carreaux_selectionnes):
#     """Exporte les données des carreaux sélectionnés dans un format facile à utiliser sur le terrain"""
    
#     print("Exportation des données pour le porte-à-porte...")
    
#     # Conversion en WGS84 pour compatibilité avec applications mobiles
#     carreaux_selectionnes_wgs84 = carreaux_selectionnes.to_crs("EPSG:4326")
    
#     # Ajout de colonnes avec les coordonnées du centre pour faciliter la navigation
#     carreaux_selectionnes_wgs84['centre_lon'] = carreaux_selectionnes_wgs84.geometry.centroid.x
#     carreaux_selectionnes_wgs84['centre_lat'] = carreaux_selectionnes_wgs84.geometry.centroid.y
    
#     # Sélection des colonnes utiles pour le terrain
#     colonnes_utiles = [
#         'IdINSPIRE', 'Ind', 'Men', 'densite_pop', 'pct_locataires',
#         'revenu_moyen', 'pct_bas_revenus', 'pct_fam_monoparentales',
#         'indicateur_precarite', 'centre_lon', 'centre_lat', 'geometry'
#     ]
    
#     # Création d'un GeoDataFrame simplifié
#     carreaux_terrain = carreaux_selectionnes_wgs84[colonnes_utiles].copy()
    
#     # Export en GeoPackage
#     export_path = os.path.join(DATA_DIR, "carreaux_porte_a_porte.gpkg")
#     carreaux_terrain.to_file(export_path, driver="GPKG")
    
#     # Export en GeoJSON pour applications web
#     export_geojson_path = os.path.join(DATA_DIR, "carreaux_porte_a_porte.geojson")
#     carreaux_terrain.to_file(export_geojson_path, driver="GeoJSON")
    
#     # Export en CSV avec coordonnées pour utilisation hors SIG
#     export_csv_path = os.path.join(DATA_DIR, "carreaux_porte_a_porte.csv")
#     carreaux_terrain.drop(columns=['geometry']).to_csv(export_csv_path, index=False, sep=';')
    
#     print(f"Données exportées pour le porte-à-porte : {export_path}")
    
#     # Création d'une liste d'adresses pour le porte-à-porte
#     print("URLs Google Maps pour accéder aux carreaux sélectionnés :")
#     for idx, row in carreaux_terrain.iterrows():
#         lat, lon = row['centre_lat'], row['centre_lon']
#         gmaps_url = f"https://www.google.com/maps?q={lat},{lon}"
#         print(f"Carreau {row['IdINSPIRE']} (Indice de précarité: {row['indicateur_precarite']:.2f}) : {gmaps_url}")

In [ ]:

#telecharger_donnees_insee()# Charger les données (métropole) carroyées depuis le fichier geopackage local
#carreaux_gdf = charger_donnees_insee_local(os.path.join(DATA_DIR, "Filosofi2019_carreaux_200m_gpkg", "carreaux_200m_met.gpkg"))
#carreaux_toulouse = carreaux_gdf[carreaux_gdf['lcog_geo'] == '31555']

#carreaux_toulouse.to_file(chemin_fichier, driver="GPKG")


In [ ]:
chemin_fichier = os.path.join(DATA_DIR, "carreaux_toulouse.gpkg")
gdf = gpd.read_file(chemin_fichier)

print("Calcul de l'indicateur de précarité...")
carreaux = calculer_indicateurs(gdf)
carreaux = normaliser_indicateurs(carreaux)
carreaux = calculer_indicateur_composite(carreaux)
carreaux = filtrer_carreaux(carreaux)
chemin_fichier = os.path.join(DATA_DIR, "carreaux_toulouse_precarite.gpkg")
carreaux.to_file(chemin_fichier, driver="GPKG")


In [ ]:
gdf = gpd.read_file(chemin_fichier)

# Vérifier et définir la projection si nécessaire
if gdf.crs is None:
    gdf.set_crs(epsg=2154, inplace=True)  # Lambert-93, projection courante en France

# Définir la palette de couleurs
#cmap = 'OrRd'  # Orange-Red, adaptée pour représenter des intensités

m = folium.Map(location=[43.6045, 1.4442], zoom_start=12, tiles=None)
folium.TileLayer(
    tiles='CartoDB positron',
    name='Fond de carte',
    control=False
).add_to(m)

colormap = cm.linear.YlOrRd_09.scale(0, 1)
colormap.caption = 'Valeur normalisée'
colormap.add_to(m)

# Chargement des données
gdf = gpd.read_file(chemin_fichier) #gpd.read_file("carreaux_toulouse_precarite.gpkg")

# Liste des variables à visualiser
variables = {
    "Indicateur de précarité": "indicateur_precarite",
    "Densité de population": "norm_densite_pop",
    "Taux de locataires": "norm_pct_locataires",
    "Taux de ménages pauvres": "norm_pct_men_pauvres",
    "Familles monoparentales": "norm_pct_fam_monoparentales"
}

for nom_couche, colonne in variables.items():
    fg = folium.FeatureGroup(name=nom_couche, show=(colonne == "indicateur_precarite"))
    folium.GeoJson(
        gdf,
        style_function=lambda feature, col=colonne: {
            'fillColor': colormap(feature['properties'][col]) if feature['properties'][col] is not None else '#gray',
            'color': 'black',
            'weight': 0.5,
            'fillOpacity': 0.7
        },
        tooltip=folium.GeoJsonTooltip(fields=[colonne], aliases=[nom_couche])
    ).add_to(fg)
    fg.add_to(m)

folium.LayerControl(collapsed=False).add_to(m)
m.save("carte_precarite_toulouse.html")

In [ ]:
gdf.keys()

In [ ]:
# # Sélection des carreaux cibles
# carreaux_selectionnes = selectionner_carreaux_cibles(carreaux_toulouse, seuil_precarite, nb_carreaux)

# # Création des cartes
# creer_cartes(carreaux_toulouse, carreaux_selectionnes, toulouse_gdf)

# # Export des données pour le terrain
# exporter_donnees_porte_a_porte(carreaux_selectionnes)

# print("\nTraitement terminé. Les données sont prêtes pour organiser le porte-à-porte.")
# print(f"Tous les fichiers ont été sauvegardés dans le répertoire '{DATA_DIR}'.")

In [ ]:
# # Exécution du script
# if __name__ == "__main__":
#     # Chemin vers le fichier geopackage téléchargé manuellement
#     # À modifier avec le chemin réel vers votre fichier
#     chemin_geopackage = "data/carreaux_insee_2019.gpkg"
    
#     # Vous pouvez ajuster ces paramètres selon vos besoins
#     main(chemin_geopackage, seuil_precarite=0.7, nb_carreaux=10)